In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# SARIMA model from statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
# Stationarity test
from statsmodels.tsa.stattools import adfuller
# To help find optimal p,d,q using auto_arima (optional)
from pmdarima import auto_arima
# For performance metrics
from sklearn.metrics import mean_squared_error

file_path = 'Lake_data_clean.xlsx'
df = pd.read_excel(file_path)
print(df.head())
print(df.info())




  Sitio Depth Group      Fecha  Profuidad (m)  Temp. (°C)  Turbidity (NTU)  \
0    SA      0-10 m 2018-01-11            4.5   20.520238         0.707748   
1    SA      0-10 m 2018-02-07            4.5   20.338267         0.552154   
2    SA      0-10 m 2018-03-13            4.5   20.690840         0.571171   
3    SA      0-10 m 2018-04-10            4.5   21.511399         0.428991   
4    SA      0-10 m 2018-05-15            4.5   22.710955         0.292084   

   DO (mg/L)  NO3 (µg/L)  PO4 (µg/L)  NH4 (µg/L)  NT (µg/l)  PT (µg/l)  \
0   6.560411    8.709623    0.340056    1.108292  57.322172   5.169241   
1   6.588031    4.526952    1.933218    0.496845  54.463139   6.432497   
2   6.615651    0.344281    3.526380    0.238063  51.604107   7.695752   
3   6.949880    0.169586    3.829325    0.626619  41.206353   7.745759   
4   7.315734    0.583857    3.766167    1.533951  27.292233   7.795767   

   Nutrient Load Index  
0            72.649384  
1            68.028983  
2          

In [43]:
# Create a dictionary to store the subsets for each Sitio and Depth Group
subset_dict = {}
unique_sites = df['Sitio'].unique()
unique_depths = df['Depth Group'].unique()

for site in unique_sites:
    for depth_grp in unique_depths:
        # 1) Filter for site & depth group
        subset = df[(df['Sitio'] == site) & (df['Depth Group'] == depth_grp)].copy()

        # 2) Convert date, sort, set as index
        subset['Fecha'] = pd.to_datetime(subset['Fecha'])
        subset.sort_values('Fecha', inplace=True)
        subset.set_index('Fecha', inplace=True)

        # 3) Separate numeric columns from non-numerics
        numeric_cols = subset.select_dtypes(include=[np.number]).columns
        df_numerics = subset[numeric_cols]  # only the numeric columns

        # 4) Resample to monthly, take mean of multiple readings
        monthly_numeric = df_numerics.resample('MS').mean()
        
        # 5) Interpolate if you want no missing months
        #    (This will fill months that had no measurement)
        monthly_numeric = monthly_numeric.interpolate(method='time')

        # 6)  Adding 'Sitio' and 'Depth Group' columns at the start of the DataFrame
        monthly_numeric.insert(0, 'Sitio', site)          # Insert at position 0 (start)
        monthly_numeric.insert(1, 'Depth Group', depth_grp)  # Insert at position 1 (after 'Sitio')

        #print(monthly_numeric)

        # 7) Store in your dictionary
        key = f"{site}_{depth_grp}"
        subset_dict[key] = monthly_numeric

        # Print or inspect the result
        #print(f"Processed subset for {key}:")
        #print(monthly_numeric, "\n")

print(subset_dict)
# Finally, check the keys of the dictionary
#print("Available keys in the subset dictionary:", subset_dict.keys())
#print("Available keys in the subset dictionary:", subset_dict.items())

{'SA_0-10 m':            Sitio Depth Group  Profuidad (m)  Temp. (°C)  Turbidity (NTU)  \
Fecha                                                                      
2018-01-01    SA      0-10 m            4.5   20.520238         0.707748   
2018-02-01    SA      0-10 m            4.5   20.338267         0.552154   
2018-03-01    SA      0-10 m            4.5   20.690840         0.571171   
2018-04-01    SA      0-10 m            4.5   21.511399         0.428991   
2018-05-01    SA      0-10 m            4.5   22.710955         0.292084   
...          ...         ...            ...         ...              ...   
2023-08-01    SA      0-10 m            4.5   24.079640         0.676412   
2023-09-01    SA      0-10 m            4.5   24.079640         0.545465   
2023-10-01    SA      0-10 m            4.5   24.042614         0.571249   
2023-11-01    SA      0-10 m            4.5   22.898175         0.597033   
2023-12-01    SA      0-10 m            4.5   22.418308         0.563850  

**1. Stationary Check**

* An Dicky-Fuller Test to determine differencing 

**2. Choose SARIMA Orders (p, d, q, P, D, Q, m)**

You can do this in two ways:

* A. Auto-ARIMA (Easiest): shown below
* B. Manual/Grid Search: 
If you want total control, you can brute-force over a grid of (p,d,q) and (P,D,Q,12), checking the AIC or BIC. This is more verbose, so I’ll just note it’s an option.

**3 .Train–Test Split**

Before fully finalizing, you often want to assess out-of-sample performance. For instance:

* Train: 2018–2022
* Test: 2023

In [ ]:

for key, monthly_df in subset_dict.items():
    # Extract the site/depth from the key if you want to display them separately
    # e.g. if you used f"{site}_{depth_grp}"
    # this split might fail if there's an underscore in the depth name,
    # so adapt as needed if your naming is more complex.
    parts = key.split("_", 1)
    site = parts[0]
    depth_grp = parts[1] if len(parts) > 1 else "Unknown"
    
    print("="*60)
    print(f"PROCESSING: Site={site}, Depth={depth_grp}")
    
    # 1) Make sure we have temperature data
    if 'Temp. (°C)' not in monthly_df.columns:
        print(f" -> No 'Temp. (°C)' column found for {key}; skipping.")
        continue
    
    # 2) Prepare the temperature series
    # Drop any remaining NaNs (after resample/interpolate, there may be none)
    temp_series = monthly_df['Temp. (°C)'].dropna()
    
    if len(temp_series) < 6:
        print(f" -> Not enough data points in {key} to run ADF test or auto_arima; skipping.")
        continue
    
    # 3) ADF Test
    adf_result = adfuller(temp_series)
    adf_stat, adf_pvalue = adf_result[0], adf_result[1]
    
    print(f" -> ADF Statistic = {adf_stat:.4f}, p-value = {adf_pvalue:.4f}")
    if adf_pvalue < 0.05:
        print("    The series appears stationary at 5% significance (no differencing needed).")
    else:
        print("    The series is likely non-stationary (differencing may be needed).")

    #Suppose you want up to 2022-12 as training, 2023+ as test
    train = temp_series.loc[:'2022-12']
    test = temp_series.loc['2023-01':]
        
    print(f" -> Train size: {len(train)}, Test size: {len(test)}")
        
    if len(train) < 6:
        print("    Not enough training data after splitting; skipping auto_arima.")
        continue

    print(" -> Running auto_arima for best model. This may take a moment...")
    auto_model = auto_arima(
        train,
        start_p=0, start_q=0,
        max_p=3, max_q=3,
        start_P=0, start_Q=0,
        max_P=2, max_Q=2,
        seasonal=True,
        m=12,       # monthly seasonality
        trace=False,
        error_action='ignore',
        suppress_warnings=True,
        stepwise=True
    )

    print(f"    Best ARIMA order: {auto_model.order}")
    print(f"    Best Seasonal order: {auto_model.seasonal_order}")

    # 6) Evaluate on Test set
    # Fit the final model on the training set
    # (auto_arima returns a fitted model, but we can call auto_model.predict)
    n_test = len(test)
    future_forecast = auto_model.predict(n_periods=n_test)

    # Compare forecast to actual
    if len(test) > 0:
        test_index = test.index
        # Convert forecast to a Series with the same index as 'test'
        forecast_series = pd.Series(future_forecast, index=test_index, name='Forecast')
        
        rmse = np.sqrt(mean_squared_error(test, forecast_series))
        print(f"    Test RMSE = {rmse:.4f}")
        
        # Optionally print or plot the last portion of the time series vs. forecast
        # (uncomment if you want to see a small textual preview)
        # print(test.tail(5))
        # print(forecast_series.tail(5))
    
    print("="*60, "\n")

PROCESSING: Site=SA, Depth=0-10 m
 -> ADF Statistic = -1.5908, p-value = 0.4882
    The series is likely non-stationary (differencing may be needed).
 -> Train size: 60, Test size: 12
 -> Running auto_arima for best model. This may take a moment...
    Best ARIMA order: (0, 1, 1)
    Best Seasonal order: (2, 1, 1, 12)
PROCESSING: Site=SA, Depth=10-30 m
 -> ADF Statistic = -0.6062, p-value = 0.8696
    The series is likely non-stationary (differencing may be needed).
 -> Train size: 60, Test size: 12
 -> Running auto_arima for best model. This may take a moment...
    Best ARIMA order: (0, 1, 1)
    Best Seasonal order: (2, 1, 0, 12)
PROCESSING: Site=SA, Depth=30+ m
 -> ADF Statistic = -4.5847, p-value = 0.0001
    The series appears stationary at 5% significance (no differencing needed).
 -> Train size: 60, Test size: 12
 -> Running auto_arima for best model. This may take a moment...
    Best ARIMA order: (0, 1, 0)
    Best Seasonal order: (0, 0, 0, 12)
PROCESSING: Site=WG, Depth=0-10